## Problems that a Particle filter is good for

**multimodal**: tracking more than one object simultaneously
**occlusions**: an object hiding behind another
**nonlinear behavior**: in both measurements and system
**non-gaussian noise**: in computer vison the algorithm can mistake a part of the background for part of the object
**continous**: can use continous functions
**multivariate**: can track several attributes of the state
**unknown process model**: can track an unknown process model

**Where these other methods fail**:

**Discrete bayes filter**: has most of these attributes but is discrete and univariate

**Kalman**: Only works for unimodal, linear systems with gaussian noise

**UKF**: Not multimodal, and can't handle occlusions. Good for fairly non-gaussian noise but not good when its very non-gaussian

**EKF**: Same as UKF, just more sensitive to strong linearities and non-gaussian noise

## Generic Particle Filter Algorithm

1. **Randomly Generate a bunch of particles**
These particles can have whatever state variables you need to measure

2. **Predict the next state of the particles**
Move particles based on how you predict the real system is behaving

3. **Update**
Update the weighing of the particles based on the measurement. particles that closely match the measurements are weighted higher than the particles which dont match the measurements very well.

4. **Resample**
Discard highly improbably particles and replace them with copies of the more probable particles.

5. **Compute Estimate**
Can compute a weighted mean and covariance of the set of particles to get a state estimate


- Ideally the weight of each particle would be the probability that it represents the true position of the vehicle. The probability is rarely computable but we only require that the weights are proportional to the probability.

**Predict Step**:
Usually we can use the commands that are sent to the vehicle, or some equation describing the movement that is being observed. The thing to note is that these commands and equations need to have a bit of simulated noise in order to give the particles a chance to notice any non-linear behavior

**Update Step**: The update step implements Bayes Theorem. So assign a probability to each position called the $prior$, when a new measurement comes in we multiply the current probability of that position (prior) by the $likelihood$ that the measurement matched that location

$$\begin{aligned}P(x \mid z) &= \frac{P(z \mid x)\, P(x)}{P(z)} \\
 &= \frac{\mathtt{likelihood}\times \mathtt{prior}}{\mathtt{normalization}}\end{aligned}$$

This part of the algorithm is called the Sequesntial Importance Sampling. The equations for the weights is called the importance density. The weights are the likelihood in bayes theorem

## Particle Resampling

### Degeneracy Problem

The algorithm above SIS (Sequential Importance Sampling), suffers from a $degeneracy problem$. Which basically means that over time as the filter runs, any particle that is not near the measurements will acquire an extremely low weight, meaning that the system will be wasting memory on thousands of samples that do not contribute towards a better estimate of the thing we are trying to track.

To deal with these we use a resampling algorithm. Algorithms dump any particles with a certain low weight then replace them with copies of particles with higher weight. These particles are  slightly different from the particles they copied from because the predict step injects some noise.

**determining when to resample**: Don't resample every epoch but rather, we resample by using the $effective N$ 

$$\hat{N}_\text{eff} = \frac{1}{\sum w^2}$$

So if the effective N falls below some threshhold we call the resampling algorithm. Usuall N/2, where N is the number of particles.

### Effect of sensor error on the particle filter

Opposed to the Kalman filter, the particle filter has the posibility of performing poorly if the standard deviation in the measurements is very small. This is due to how the particles are generated, if the standard deviation is 0.1, a robot is at position (1,1) and a particle is generated at (2,2) then the particle is 14 standard deviations away giving it a very small probability of being selected. So a very good sensor could actually hurt the estimate.

**some fixes to this problem**: 
    - artificially increase the sensor noise std deviation so the particle filter algorithm will accept more points as matching the robots probability distribuition. But this is not optimal because youre creating a bunch of points that are not a good match.
    
    - use more samples. this works great, but comes at the expense of using a lot of memory.
    
    - Best thing is knowing the vehicles starting position, it gives the filter an easier time. But don't be too exact because if the particles are not disperced enough the filter might not capture the non-linearities of the problem.

### Importance Sampling

So it might be hard to actually figure out the probability distribuition that describes the position and movement of a robot to compute an integral using monte carlo methods. If our state model is incorrect we can still weight the samples based off the probability distribuition that we interested in. This is effectively what happens when you weight the samples based off the measurement information. So our state model can be a bit off, because the measurement information will resemble the actual probablily distribuition of what we're trying to model.